In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g4j-_zqZ1BkIwpBg_4eJNPKWo8K6y588X6Kj71IRx8yjq93mwkPPXc
Mounted at /content/gdrive


In [ ]:
import numpy as np
import os
import cv2
from tqdm import tqdm

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
DATADIR = r"/content/gdrive/My Drive/Weather"

CATEGORIES = ['Cloudy', 'Rain', 'sunny', 'sunset']

IMG_SIZE =150
img_rows, img_cols = 150,150
training_data = []

def create_training_data():
    for category in CATEGORIES:
        
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num]) 
            except Exception as e:
                pass

create_training_data()

print(len(training_data))

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

100%|██████████| 358/358 [02:16<00:00,  2.62it/s]

1123


In [ ]:
X1 = np.array(X)
X1.shape

(1123, 150, 150, 3)

In [ ]:
y = np.array(y)
y.shape

(1123,)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=4)
X_train = X_train.reshape(X_train.shape[0],img_rows, img_cols,3)
X_test = X_test.reshape(X_test.shape[0],img_rows, img_cols,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, 4)
Y_test = np_utils.to_categorical(y_test, 4)

In [ ]:
X_test.shape

(225, 150, 150, 3)

In [ ]:
Y_train.shape

(898, 4)

In [ ]:
input_shape = [150, 150, 3]
model = Sequential()
model.add(DenseNet169(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=4,
))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(units=4,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Functional)     (None, 4, 4, 1664)        12642880  
_________________________________________________________________
flatten (Flatten)            (None, 26624)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              27264000  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4100      
Total params: 39,910,980
Trainable params: 39,752,580
Non-trainable params: 158,400
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train,Y_train,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 262s 11s/step - loss: 9.9193 - accuracy: 0.4358 - val_loss: 59309.9883 - val_accuracy: 0.1667
Epoch 2/10
23/23 [==============================] - 244s 11s/step - loss: 2.7232 - accuracy: 0.6771 - val_loss: 1.0380 - val_accuracy: 0.6000
Epoch 3/10
23/23 [==============================] - 244s 11s/step - loss: 0.6156 - accuracy: 0.7516 - val_loss: 1.5430 - val_accuracy: 0.3056
Epoch 4/10
23/23 [==============================] - 247s 11s/step - loss: 0.5835 - accuracy: 0.7915 - val_loss: 2.3865 - val_accuracy: 0.2944
Epoch 5/10
23/23 [==============================] - 243s 11s/step - loss: 0.4841 - accuracy: 0.8103 - val_loss: 2.4611 - val_accuracy: 0.3000
Epoch 6/10
23/23 [==============================] - 245s 11s/step - loss: 0.4232 - accuracy: 0.8396 - val_loss: 2.1074 - val_accuracy: 0.3056
Epoch 7/10
23/23 [==============================] - 244s 11s/step - loss: 0.4001 - accuracy: 0.8455 - val_loss: 1.8198 - val_accuracy: 0.3389
Ep

In [ ]:
scores =model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 53.45%


In [ ]:
model.save("/content/gdrive/My Drive/model/weather_densenet.h5")
print("Saved model to disk")

Saved model to disk
